# 1 Understanding the binary heap structure

We studied the Heap data structure in this week's lecture. A heap is typically implemented as an array, where the root is placed at index 1 (rather than 0, for convenience). It implements the ADT "priority queue".

## 1. What is the array representation of this binary heap:

![alt text](https://i.loli.net/2019/09/24/DjP6Crnc4FoubNe.png)

`[7, 10, 8, 14, 13, 12, 11, 15, 20, 25]`

## 2, Recall how for a given node of index $i$, its parent, left child and right child can be accessed (if they exist). Are those operations cheap on a computer?

Yes. Continuous linear data structure costs $O(1)$ to access any element.

Also, when we access an element in the array(list), we may have the cache of the list.

**From solution:**
List representation uses index to access each element. They have these relationship:
* The index of a node's parent: `i//2`
* The index of a node's left child: `i*2`
* The index of a node's right child: `i*2+1`

In modern computer, the division and multiplication can be done by bit shifting.

In [4]:
# Bit shifting vs algebra computation
import time

st = time.time()
for _ in range(1145141919):
    114514*2
ed = time.time()
print("Algebra multiplication: %.2f"%(ed-st))
st = time.time()
for _ in range(1145141919):
    114514<<1
ed = time.time()
print("Shifting multiplication: %.2f"%(ed-st))

Algebra multiplication: 41.20
Shifting multiplication: 41.07


## 3. Suppose we are given the list `[10, 14, 8, 7, 16, 9, 2, 4, 1]`, give a min-heap with this list. Are there multiple valid heaps?

**YES**, there are multiple valid heaps. Since the min-heap has a property that the each item is always the minimum element in the tree where that item is the root.

## 4. In a min-heap, at what node(s) can be second smallest number be? What about the third smallest number? Can we say anything about the largest number?

General speaking, the second smallest number can be found in height 1. But the third smallest number can be found in 2nd or 3rd level. A min-heap does not guarantee that all the numbers in any level are greater than the ones in upper level. However, we can say the largest number in the deepest level since it could not be a root of any sub-heap. It must reside in the final level as a leaf.

## 5. How many items does a heap of height $h$ store? (give an interval)
$2^h$ ~ $2^{h+1} - 1$, where $n$ is the number of levels.

A heap is a binary tree. For a tree with height $h-1$, it has total number of nodes: $1 + 2 + 4 + 8 + \dots + 2^{i-1}$, where $i = h + 1$. # of nodes $2^h-1$ of height $h-1$. Having considered that it might not be a full tree. A tree with height $h$ has a maximum number of $2^{h+1}-1$ nodes.

Therefore, the interval is $2^h$~$2^{h+1}-1$

# 2 $k^{th}$ smallest in a min-heap

## 1. At what level(s) could the third smallest element be?

Level 1 or 2.

*NB: This unit has a convention that the root is level 0.* 

## 2. At what level(s) could the maximum element be?

The bottom level if it was a min-heap.

The top level if it was a max-heap.

## 3. Under what condition(s) on $k$ can we guarantee that the $k^{th}$ smallest element is not at a leaf?

$k < \log n$

Assuming that $k^{th}$ elements distribute averagely in each level, each of each has one except the leaf level. Considering the height is $h = \log n$, k must comply the condition $k \lt \log n$

## 4. Under what condition(s) on $k$ can we guarantee that the $k^{th}$ smallest element is not at level 1, under the root?

$ k > 3$

# 3 Verifying a binary heap

In this question you may suppose that the input list starts with a dummy item (e.g. 0) at index 0.

### 1. Write an interative algorithm that outputs `True` if and only if a given list is a binary heap, without using methods or code of the class `BinHeap`.

In [0]:
def isBinHeap(h, i=1):
    if (2*i) <= len(h) or (2*i + 1) < len(h):
        currKey = h[i]
        if (2*i + 1) < len(h):
            leftChildKey = h[2*i]
            rightChildKey = h[2*i+1]
            if currKey > leftChildKey or currKey > rightChildKey:
                return False
            return isBinHeap(h, 2*i) and isBinHeap(h, 2*i+1)
        else:
            leftChildKey = h[2*i]
            if currKey > leftChildKey:
                return False
            return isBinHeap(h, 2*i)
    return True

def isBinHeapIter(h):
    i = 1
    while i*2+1 < len(h):
        if h[i] > h[i*2] or h[i] > h[i*2+1]:
            return False
        else:
            i += 1
    return True

In [0]:
h1 = [0, 7, 10, 8, 14, 13, 12, 11, 15, 20, 25]
h2 = [0, 10, 14, 8, 7, 16, 9, 2, 4, 1]
assert isBinHeapIter(h1) is True
assert isBinHeapIter(h2) is False

### 2. What is the complexity of this algorithm?

The worst case is where given any node, its children have keys that are greater than or equal to its. Except the recursion statements, all other statements cost constant time. Using telescoping:

$T(n) = 2T(n-1) + c$

$T(n) = 2(2T(n-1-2) + c)+c = 4T(n-1-2) + (1+2)c$

$T(n) = 2(2(2T(n-1-2-4) + c) + c)+c = 8T(n-1-2-4) + (1+2+4)c$

$\dots$

While finishing $k$ level , we have:

$T(n) = 2T(n - (2^k+1)) + (2^k+1)c$

Termination condition: reaching the leaf

$k = \log n$

$T(n) = (2^{\log n}+1)c = (n+1)c => O(n)$

# 4 Maintaining the binary heap structure

In [0]:
def percDown(h, i=1):
    while (i*2) < len(h) - 1:
        mc = minChild(h, i)
        if h[i] > h[mc]:
            tmp = h[i]
            h[i] = h[mc]
            h[mc] = tmp
        i = mc

def minChild(h, i):
    if (i*2) + 1 < len(h) - 1:
        return (i*2) if h[i*2] < h[i*2+1] else (i*2+1)
    else:
        return (i*2)

Given an index $i$, the `percDown` function ensures that the tree rooted at the node of index $i$ maintians the min-heap property, supposing that this property is satisfied for the children of $i$. In this exercise, we ask you to write a recursive function `percDownRecursively` that does the same `percDown`. Answering those questions first may help you:

### 1. What is/are the base case(s) of the recursion, i.e. when is there no need to make a recursive call?

Reached the leaf node where there is no child to percollate.



### 2. What is the maximum number of recursive calls that are made at each call?

$\log n$

### 3. Write the function percDownRecursively as part of the class BinHeap given in the online book.

In [0]:
class BinHeap:
    def __init__(self):
        self.heapList = [0]
        self.currentSize = 0

    def percUp(self,i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i // 2]:
                tmp = self.heapList[i // 2]
                self.heapList[i // 2] = self.heapList[i]
                self.heapList[i] = tmp
            i = i // 2

    def insert(self,k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def percDown(self,i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc

    def minChild(self,i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i*2] < self.heapList[i*2+1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMin(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)
        return retval

    def buildHeap(self,alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while (i > 0):
            self.percDownRecursive(i)
            i = i - 1

    # TODO here
    def heapSort(self, l):
        self.buildHeap(l)
        sortedl = []
        while self.currentSize != 0:
            sortedl.append(self.delMin())
        return sortedl

    def percDownRecursive(self, i):
        if i*2 <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            self.percDownRecursive(mc)

### 4. Test percDownRecursively by using it in buildHeap on the following tree, with i = 0:

In [20]:
th = [8, 16, 10, 14, 7, 9, 3, 2, 4, 5]
h = BinHeap()
h.buildHeap(th)
print(h.heapList)


[0, 2, 4, 3, 8, 5, 9, 10, 14, 16, 7]


In [19]:
h = BinHeap()
h.buildHeap([8, 16, 10, 14, 7, 9, 3, 2, 4, 5])
print(h.heapList)

[0, 2, 4, 3, 8, 5, 9, 10, 14, 16, 7]


# 5 Sorting with a heap: heapsort

## 1. Write a `heapSort` function that sorts a list using the calss `BinHeap`

In [0]:
class BinHeap:
    def __init__(self):
        self.heapList = [0]
        self.currentSize = 0

    def percUp(self,i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i // 2]:
                tmp = self.heapList[i // 2]
                self.heapList[i // 2] = self.heapList[i]
                self.heapList[i] = tmp
            i = i // 2

    def insert(self,k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def percDown(self,i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc

    def minChild(self,i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i*2] < self.heapList[i*2+1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMin(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)
        return retval

    def buildHeap(self,alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1

    # TODO here
    def heapSort(self, l):
        self.buildHeap(l)
        sortedl = []
        while self.currentSize != 0:
            sortedl.append(self.delMin())
        return sortedl

In [0]:
l = [1, 4, 2, 6, 5, 12, 7, 9, 6, 3, 5, 24]
h = BinHeap()
print(h.heapSort(l))

[1, 2, 3, 4, 5, 5, 6, 6, 7, 9, 12, 24]


## 2. Using the solutions to Week 7, compare the running time of `heapSort` with other sorting algorithms.

Process:

1. Build the heap using input list.
2. Keep popping the root and percollating down.
3. Return the sorted list.

Step by step analysis.

Building heap: $O(n)$

Popping and Percollating down: 

1. $O(\log n)$ per popping.

2. n elements to delete

    ==> $O(n\log n)$.

Return: $O(1)$

Therefore, it's $O(n\log n)$.

# 6 $k$ smallest elements

## 1. Explain how the $k$ smallest elements from an unordered list of size $n$ can be found in time $O(n + k\log n)$ using a min-heap.

1. building a heap with given list. $O(n)$
2. Iteratively delete the min for $k$ times. $O(k\log n)$, delMin cost $O(\log n)$.

## 2. Explain how the $k$ smallest elements from an unordered list of size $n$ can be found in time $O(n\log k)$ using $O(k)$ auxiliary space.

1. Defining a heap with size $k+1$. $O(k)$
2. Filling the heap with the first $k$ elements. $O(k\log k)$
3. Iterate remaining items and put them into the heap one by one. During this process, the heap will accept the inserted elements and put it at the end, then, percUp it to the appropriate position. $O((n-k)\log k)$
4. Pop out the min from heap. $O(k\log k)$.

From step 3, it can be found that the time complexity is determined by $O(n\log k)$